In [1]:
import time
import numpy as np
import types
import os
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from scipy.special import zeta
import PRyM.PRyM_init as PRyMini
import PRyM.PRyM_thermo as PRyMthermo
import PRyM.test_main3 as test_main
#import PRyM.PRyM_jl_sys as PRyMjl
from numdifftools import Derivative
import PRyM.PRyM_eval_nTOp as PRyMevalnTOp
import PRyM.PRyM_nTOp as PRyMnTOp
import matplotlib.pyplot as plt
import pandas as pd
import random

In [2]:
#observed values, all wrong currently, for testing purposes
YPCMB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs = np.array([0.245,3.85,2.547,1.63])
X_obs = np.array([YPCMB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs])
YPCMB_sig, Neff_sig, DoHx1e5_sig, Li7oHx1e10_sig = np.array([0.003,0.62,0.0029,0.3])
sigma_X = np.array([YPCMB_sig, Neff_sig, DoHx1e5_sig, Li7oHx1e10_sig])

eta_obs = 6.04*10**(-10)
eta_sigma = 0.118*10**(-10)


In [3]:
#vary 12 key nuclear rates
#params for standard normal 
mu = 0       # Mean
sigma = 1    # Standard deviation
#choose a random sample p using standard normal rate = median_rate*exp(p sigma)
PRyMini.p_npdg = np.random.normal(loc=mu, scale=sigma)
PRyMini.p_dpHe3g = np.random.normal(loc=mu, scale=sigma)
PRyMini.p_ddHe3n = np.random.normal(loc=mu, scale=sigma)
PRyMini.p_ddtp = np.random.normal(loc=mu, scale=sigma)
PRyMini.p_tpag = np.random.normal(loc=mu, scale=sigma)
PRyMini.p_tdan = np.random.normal(loc=mu, scale=sigma)
PRyMini.p_taLi7g = np.random.normal(loc=mu, scale=sigma)
PRyMini.p_He3ntp = np.random.normal(loc=mu, scale=sigma)
PRyMini.p_He3dap = np.random.normal(loc=mu, scale=sigma)
PRyMini.p_He3aBe7g = np.random.normal(loc=mu, scale=sigma)
PRyMini.p_Be7nLi7p = np.random.normal(loc=mu, scale=sigma)
PRyMini.p_Li7paa = np.random.normal(loc=mu, scale=sigma)

#theoretical nuisance parameters
#p is the nuisance parameter which is chosen from standard normal
nus_th = np.array([PRyMini.p_npdg, PRyMini.p_dpHe3g, PRyMini.p_ddHe3n, PRyMini.p_ddtp, PRyMini.p_tpag, PRyMini.p_tdan, PRyMini.p_taLi7g, PRyMini.p_He3ntp, PRyMini.p_He3dap, PRyMini.p_He3aBe7g, PRyMini.p_Be7nLi7p, PRyMini.p_Li7paa])
#observed nuisance parameters
nus_obs = np.zeros(len(nus_th))
#corresponding sigmas for nuisance parameters
sigma_nus = np.array([1 for n in range(len(nus_obs))])



In [4]:
#observed values
YPCMB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs = np.array([0.245,3.85,2.547,1.63])
X_obs = np.array([YPCMB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs])
YPCMB_sig, Neff_sig, DoHx1e5_sig, Li7oHx1e10_sig = np.array([0.003,0.62,0.0029,0.3])
sigma_X = np.array([YPCMB_sig, Neff_sig, DoHx1e5_sig, Li7oHx1e10_sig])

#eta_b experimental values today
eta_obs = 6.04*10**(-10)
sigma_eta = 0.118*10**(-10)





In [ ]:
# Chi^2 function

def chi2_fn(params):
    #initialize stasis parameters
    PRyMini.compute_bckg_flag = True
    PRyMini.save_bckg_flag = True
    PRyMini.stasis_flag = True
    PRyMini.verbose_flag = False
    PRyMini.smallnet_flag = True
    PRyMini.stasis_params['gamma'] = 1
    PRyMini.stasis_params['delta'] = 1
    PRyMini.stasis_params['alpha'] = params[0]
    PRyMini.stasis_params['m0'] = params[1]
    PRyMini.stasis_params['Delta_m'] = params[1]
    PRyMini.stasis_params['plasma_inject_frac'] = params[2]
    PRyMini.stasis_params['tiny_radiation'] = 0.1
    PRyMini.stasis_params['stasis_end_mev'] = params[3]
    PRyMini.stasis_params['N_SPECIES'] = 25

    #calculate theoretical values
    run = test_main.PRyMclass()
    results = run.PRyMresults()

    ##extract etab today
    #extract lowest temperature and corresponding radiation density 
    path_stasis = "/Users/alechewitt/Desktop/Stasis/Stasis1/PRyMrates/thermo/stasis_abundances.txt"
    path_stasis = "/pub/ahewitt1/Stasis_new/Stasis1/PRyMrates/thermo/stasis_abundances.txt"
    with open(path_stasis, "r") as f:
        lines = f.readlines()
        last_line = lines[-1].strip()
        last_line = np.array([float(x) for x in last_line.split()])
    kb = 1      #I think this is right in natural units
    T_MeV = last_line[1]
    rho_g = last_line[3]
    n_g = rho_g/T_MeV
    nB = run.nB(run.a_of_T(T_MeV))
    eta_th = nB/n_g

    Neff, Omeganurel, OneOverOmeganunr, YPCMB, YPBBN, DoHx1e5, He3oHx1e5, Li7oHx1e10 = results
    #X_th = np.array([Neff, Omeganurel, OneOverOmeganunr, YPCMB, YPBBN, DoHx1e5, He3oHx1e5, Li7oHx1e10])
    X_th = np.array([YPCMB, Neff, DoHx1e5, Li7oHx1e10])
    return sum(((X_th-X_obs)/sigma_X)**2) + ((eta_th-eta_obs)/sigma_eta)**2 + sum(((nus_th-nus_obs)/sigma_nus)**2)

def chi2_fn_test(params):
    #initialize stasis parameters
    PRyMini.compute_bckg_flag = True
    PRyMini.save_bckg_flag = True
    PRyMini.stasis_flag = True
    PRyMini.verbose_flag = False
    PRyMini.smallnet_flag = True
    PRyMini.stasis_params['gamma'] = 1
    PRyMini.stasis_params['delta'] = 1
    PRyMini.stasis_params['alpha'] = params[0]
    PRyMini.stasis_params['m0'] = params[1]
    PRyMini.stasis_params['Delta_m'] = params[1]
    PRyMini.stasis_params['plasma_inject_frac'] = params[2]
    PRyMini.stasis_params['tiny_radiation'] = 0.1
    PRyMini.stasis_params['stasis_end_mev'] = params[3]
    PRyMini.stasis_params['N_SPECIES'] = 25

    #calculate theoretical values
    run = test_main.PRyMclass()
    results = run.PRyMresults()

    ##extract etab today
    #extract lowest temperature and corresponding radiation density 
    path_stasis = "/Users/alechewitt/Desktop/Stasis/Stasis1/PRyMrates/thermo/stasis_abundances.txt"
    path_stasis = "/pub/ahewitt1/Stasis_new/Stasis1/PRyMrates/thermo/stasis_abundances.txt"
    with open(path_stasis, "r") as f:
        lines = f.readlines()
        last_line = lines[-1].strip()
        last_line = np.array([float(x) for x in last_line.split()])
    kb = 1      #I think this is right in natural units
    T_MeV = last_line[1]
    rho_g = last_line[3]
    n_g = rho_g/T_MeV
    nB = run.nB(run.a_of_T(T_MeV))
    eta_th = nB/n_g

    Neff, Omeganurel, OneOverOmeganunr, YPCMB, YPBBN, DoHx1e5, He3oHx1e5, Li7oHx1e10 = results
    #X_th = np.array([Neff, Omeganurel, OneOverOmeganunr, YPCMB, YPBBN, DoHx1e5, He3oHx1e5, Li7oHx1e10])
    X_th = np.array([YPCMB, Neff, DoHx1e5, Li7oHx1e10])
    return sum(((X_th-X_obs)/sigma_X)**2), ((eta_th-eta_obs)/sigma_eta)**2, sum(((nus_th-nus_obs)/sigma_nus)**2)

# Gradient of chi^2 (numerical)
def grad_chi2(chi2_fn,params, eps=1e-3):
    grad = np.zeros_like(params)
    for i in range(len(params)):
        dparams = np.zeros_like(params)
        dparams[i] = eps
        grad[i] = (chi2_fn(params + dparams) - chi2_fn(params - dparams)) / (2 * eps)
        if i==2:
            print('mass before: ',params[i],' mass after: ',params[i]+dparams[i],'. These should be different. gradient: ',grad[i])
            print('chisq(p+dp) = ',chi2_fn(params + dparams),'chisq(p-dp) = ',chi2_fn(params - dparams),' eps = ',eps)
    return grad


In [6]:
#adaptive gradient method
def adagrad_minimize(chi2_fn, params_init, lr=0.0001, epsilon=1e-8, max_iters=1000, tol=1e-6, verbose=True):
    params = np.array(params_init, dtype=np.float64)
    G = np.zeros_like(params)
    
    for t in range(1, max_iters + 1):
        grad = grad_chi2(chi2_fn, params)
        G += grad ** 2
        adjusted_grad = grad / (np.sqrt(G) + epsilon)
        update = lr * adjusted_grad
        params -= update
        params[-1] = int(np.floor(params[-1]))#N_species needs to be an integer
        learning_rate = lr / (np.sqrt(G) + epsilon)
        if verbose:
            print(f"Iter {t}: params = {params}, chisq = {chi2_fn(params):.6f}, grad = {grad}, |grad| = {np.linalg.norm(grad):.3e}, learning rate*grad = {update}")
        
        if np.linalg.norm(update) < tol:
            break
            
    return params

In [7]:
PRyMini.stasis_params

{'gamma': 7,
 'delta': 1,
 'm0': 1,
 'Delta_m': 1,
 'N_SPECIES': 10,
 'GAMMA_OVER_H': 100000.0,
 'stasis_end_mev': 50,
 'tiny_radiation': 0.01,
 'plasma_inject_frac': 0.1,
 'alpha': 1.0}

In [ ]:
#run stasis gradient descent and save results ot a file
# Optional: write header only once
file_name = "results_grad_descent.txt"
with open(file_name, "w") as ff:
    ff.write("gamma delta alpha delm m0 fplas f Tend Nspecies chi2\n")
#initialize random stasis parameters in the desired intervals
gamma = 1
delta = 1
alpha = random.uniform(-1/delta,(gamma/2)-(1/delta))
delm = random.uniform(1,10)
m0 = delm
fplas = random.uniform(0.1,1)
f = 0.1
Tend = random.uniform(0.1, 10)
#Nspecies = int(random.randint(1, 20)) #old
Nspecies = int(25)

#eta0b
PRyMini.eta0b = random.uniform(1e-11,1e-9 )

#initialize params
params = [alpha, delm, fplas, Tend]
params = np.array(params)
print(params)

print(chi2_fn_test(params))

#adagrad_minimize(chi2_fn, params, lr=0.1, epsilon=1e-4, max_iters=100, tol=1e-6, verbose=True)

[-0.55188273  2.48499905  0.55745186  6.30908433]


mass before:  0.5574518590185042  mass after:  0.5584518590185042 . These should be different. gradient:  -50022.9616977158
chisq(p+dp) =  149533.23973536093 chisq(p-dp) =  149633.28565875636  eps =  0.001


KeyboardInterrupt: 

In [ ]:
PRyMini.stasis_params

{'gamma': 1,
 'delta': 1,
 'm0': np.float64(8.160661774678584),
 'Delta_m': np.float64(8.160661774678584),
 'N_SPECIES': 25,
 'GAMMA_OVER_H': 100000.0,
 'stasis_end_mev': np.float64(3.7305774335548914),
 'tiny_radiation': 0.1,
 'plasma_inject_frac': np.float64(0.5998851493142416),
 'alpha': np.float64(-0.7348253362819448)}

In [ ]:
#minimize -2 ln L with a package
from scipy.optimize import minimize
import numpy as np

'''#initialize random stasis parameters in the desired intervals
gamma = random.uniform(4,8)
delta = random.uniform(0,3)
alpha = (2/7)*gamma - (1/delta)
delm = random.uniform(1,10)
m0 = delm
fplas = random.uniform(0.1,1)
f = random.uniform(0.1,1)
Tend = random.uniform(1, 10)
Nspecies = int(random.randint(1, 20))
#PRyMini.stasis_params
print(Nspecies)
#initialize params
params = [gamma,delta,alpha,delm,m0,fplas,f,Tend,Nspecies]

# Initial guess
#x0 = np.array([0.0, 0.0])

# Minimize
res = minimize(chi2_fn, params, method='BFGS')'''

19
{'gamma': np.float64(4.733232563550961), 'delta': np.float64(2.413703625087335), 'm0': np.float64(8.72149004071636), 'Delta_m': np.float64(8.72149004071636), 'N_SPECIES': 19, 'GAMMA_OVER_H': 100000.0, 'stasis_end_mev': np.float64(1.3619708154105123), 'tiny_radiation': np.float64(0.9053552727974801), 'plasma_inject_frac': np.float64(0.22748234321992722), 'alpha': np.float64(0.9380510887510518)}
{'gamma': np.float64(4.733232578452122), 'delta': np.float64(2.413703625087335), 'm0': np.float64(8.72149004071636), 'Delta_m': np.float64(8.72149004071636), 'N_SPECIES': 19, 'GAMMA_OVER_H': 100000.0, 'stasis_end_mev': np.float64(1.3619708154105123), 'tiny_radiation': np.float64(0.9053552727974801), 'plasma_inject_frac': np.float64(0.22748234321992722), 'alpha': np.float64(0.9380510887510518)}
{'gamma': np.float64(4.733232563550961), 'delta': np.float64(2.4137036399884964), 'm0': np.float64(8.72149004071636), 'Delta_m': np.float64(8.72149004071636), 'N_SPECIES': 19, 'GAMMA_OVER_H': 100000.0, '

KeyboardInterrupt: 

In [ ]:
'''import numpy as np
import emcee
import corner
import matplotlib.pyplot as plt

# ----- Define the true parameters -----
true_mean = np.array([0.0, 0.0])
true_cov = np.array([[1.0, 0.8], [0.8, 1.5]])

# ----- Log-probability function -----
def log_prob(theta):
    diff = theta - true_mean
    inv_cov = np.linalg.inv(true_cov)
    return -0.5 * diff @ inv_cov @ diff

# ----- MCMC setup -----
ndim = 2              # number of parameters
nwalkers = 32         # number of walkers
nsteps = 2000         # number of steps per walker
initial_pos = np.random.randn(nwalkers, ndim)  # starting guess

# ----- Run the sampler -----
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob)
sampler.run_mcmc(initial_pos, nsteps, progress=True)

# ----- Get the samples -----
samples = sampler.get_chain(discard=500, thin=10, flat=True)  # burn-in and thinning

# ----- Make a corner plot -----
fig = corner.corner(samples, labels=["$x$", "$y$"], truths=true_mean)
plt.show()'''